# ConvNet con Keras
In questo notebook vederemo come possiamo creare una rete neurale convoluzionale (ConvNet) utilizzando Keras. Lo scopo della nostra rete sarà quello di riconoscere capi di abbigliamento, calzature e accessori utilizzando sempre il Fashion-MNIST. Al termine del notebook confronteremo il risultato ottenuto con quello [già ottenuto con un più semplice percettrone multistrato](http://localhost:8888/notebooks/6%20-%20Overfitting%20e%20metodi%20di%20regolarizzazione/Regolarizzazione%20L1%20e%20L2.ipynb).
<br><br>
Cominciamo ad importare i moduli che ci serviranno.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

from keras.utils import to_categorical

Carichiamo il Fashion-MNIST all'interno di due coppie di array numpy, per addestramento e test, utilizzando direttamente Keras.

In [44]:
from keras.datasets import fashion_mnist

labels = ["T-shirt/top","Pantalone","Pullover","Vestito","Cappotto","Sandalo","Maglietta","Sneaker","Borsa","Stivaletto"]

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

print("Numero totale di proprietà: "+str(X_train.shape[1]))
print("Esempi di training: "+str(X_train.shape[0]))
print("Esempi di test: "+str(X_test.shape[0]))

Numero totale di proprietà: 28
Esempi di training: 60000
Esempi di test: 10000


Normalizziamo i dati

In [ ]:
X_train = X_train/255
X_test = X_test/255

L'input di una rete convoluzionale deve avere dimensione del tipo N*H*W*C, dove:
* N è il numero di esempi nel dataset
* H e W sono rispettivamente altezza e larghezza dell'immagine in pixel
* C sono i canali dell'immagine

Nel caso di un'immagine a colori (formato RGB) i canali saranno 3, per un immagine in bianco e nero invece abbiamo un solo canale. Utilizziamo il metodo reshape per modificare la dimensione degli array con le features, in modo da conformarci alle specifiche di Keras.

In [46]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_train.shape

(60000, 28, 28, 1)

Perfetto! Infine creiamo delle variabili di comodo per il target

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Creazione del modello

Paramteri Conv2D:
- filters: numero dei filtri che verranno applicati alle immagini
- kernel_size: dimensione dei filtri
- padding: tipi di padding da usare, same indica che la feature map avrà la stessa dimensione dell'immagine di partenza
- activation: funzione di attivazione
- input_shape: indica l'input, che sarà la dimensione delle immagini

La rete neurale in automatico sceglierà i filtri migliori.

In [ ]:
from keras.layers import Conv2D, Flatten, Dropout

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
flatten_9 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               6422656   
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1290      
Total params: 6,424,586
Trainable params: 6,424,586
Non-trainable params: 0
_________________________________________________________________


- 64*2*2+64 = 320
- 64*28*28*1 = 50176
- 50176*128+128 = 6422656
- 128*10+10 = 1290

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 6s 124us/step - loss: 0.6749 - acc: 0.7686 - val_loss: 0.3887 - val_acc: 0.8613
Epoch 2/10
48000/48000 [==============================] - 4s 88us/step - loss: 0.4055 - acc: 0.8590 - val_loss: 0.3256 - val_acc: 0.8812
Epoch 3/10
48000/48000 [==============================] - 4s 89us/step - loss: 0.3422 - acc: 0.8809 - val_loss: 0.2949 - val_acc: 0.8933
Epoch 4/10
48000/48000 [==============================] - 4s 89us/step - loss: 0.3036 - acc: 0.8930 - val_loss: 0.2790 - val_acc: 0.8980
Epoch 5/10
48000/48000 [==============================] - 4s 89us/step - loss: 0.2787 - acc: 0.9018 - val_loss: 0.2689 - val_acc: 0.9008
Epoch 6/10
48000/48000 [==============================] - 4s 89us/step - loss: 0.2534 - acc: 0.9093 - val_loss: 0.2675 - val_acc: 0.9030
Epoch 7/10
48000/48000 [==============================] - 4s 89us/step - loss: 0.2371 - acc: 0.9147 - val_loss: 0.2611 - val_ac

In [78]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 109us/step


[0.27195664439201356, 0.9024]

## Utilizzare uno strato di Pooling

In [ ]:
from keras.layers import MaxPooling2D

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [81]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 14, 14, 32)        8224      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 1568)              0         
__________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [83]:
from keras.callbacks import History, EarlyStopping 

history = History()
earlyStopping = EarlyStopping(min_delta=0.001, patience=5)


model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=100, callbacks=[history, earlyStopping])

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 6s 117us/step - loss: 0.9287 - acc: 0.6519 - val_loss: 0.5252 - val_acc: 0.8071
Epoch 2/100
48000/48000 [==============================] - 4s 84us/step - loss: 0.5399 - acc: 0.8002 - val_loss: 0.4407 - val_acc: 0.8372
Epoch 3/100
48000/48000 [==============================] - 4s 84us/step - loss: 0.4723 - acc: 0.8264 - val_loss: 0.3900 - val_acc: 0.8581
Epoch 4/100
48000/48000 [==============================] - 4s 83us/step - loss: 0.4293 - acc: 0.8456 - val_loss: 0.3594 - val_acc: 0.8703
Epoch 5/100
48000/48000 [==============================] - 4s 83us/step - loss: 0.4068 - acc: 0.8524 - val_loss: 0.3350 - val_acc: 0.8782
Epoch 6/100
48000/48000 [==============================] - 4s 84us/step - loss: 0.3883 - acc: 0.8581 - val_loss: 0.3256 - val_acc: 0.8823
Epoch 7/100
48000/48000 [==============================] - 4s 84us/step - loss: 0.3671 - acc: 0.8656 - val_loss: 0.3129 -

In [84]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 99us/step


[0.22296086776852608, 0.9207]